# 讀取健身房店名檔案 

In [7]:
import csv
with open("new_gymlist.csv", 'r', encoding = 'utf8') as in_file:
        reader = csv.reader(in_file, delimiter=',')
        my_list = list(reader)
        google_gym = my_list[0]
        
len(google_gym)

641

In [1]:
import csv
def getStoreName():
    with open("new_gymlist.csv", 'r', encoding = 'utf8') as in_file:
        reader = csv.reader(in_file, delimiter=',')
        my_list = list(reader)
        google_gym = my_list[0]
        
        for gym in google_gym:
            q.put(gym)
    

# 取得評論總數

In [2]:
def getCnt(driver):                             
    #點開評論
    firstButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"""//*[@id="rhs_block"]/div/div[1]/div/div[1]/div[2]/div[1]/div/div[2]/div[2]/div/div/span[2]/span/a/span""")))
    firstButton.click()
    #爬取評論總數
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="gsr"]/g-lightbox/div[2]/div[3]/div/div/div/div[1]/div[3]/div[1]/div/span')))
    comment_number_path = driver.find_element_by_xpath('//*[@id="gsr"]/g-lightbox/div[2]/div[3]/div/div/div/div[1]/div[3]/div[1]/div/span')
    comment_cnt=int(comment_number_path.text.split(" ")[0])
    
    return comment_cnt


# 取得所有評論內容

In [3]:
import math
def loadComment(driver,comment_cnt):
    #下拉評論
    for n in range (1,math.floor(comment_cnt/10)+2):
        try:
            next_ele_xpath = str('''//*[@id="reviewSort"]/div[{}]/div[3]/div[10]/div[1]/div[3]/div[2]/span'''.format(n))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_ele_xpath)))
            show_next_comment = driver.find_element_by_xpath(next_ele_xpath)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, next_ele_xpath)))
            driver.execute_script("""return arguments[0].scrollIntoView();""", show_next_comment)
        except TimeoutException as err:
            print (str(err))
    print("拉了"+str(n)+"次")
    
    #抓取所有內容
    html = driver.page_source
    soup = BeautifulSoup(html,"lxml")
    
    return(soup)

# 取得所有資料 

In [4]:
def getData(soup):
    reviews={}
    reviews = soup.find_all('div', class_='_vor')
    gym_dict = {}
    gym = []

    #抓店名
    gym_dict['store_name'] = soup.find_all('div', class_ ='_h8k')[0].text

    #抓地址
#     gym_dict['addr'] = driver.find_element_by_css_selector("""#gsr > g-lightbox > div._Nzh._oAh._xop.iNFfR_ZJk_ds-20H57zYdxbY > div._Azh.iNFfR_ZJk_ds-0078sLar460 > div > div > div > div.fp-w.review-dialog-top > div._Z7k > div._K7k > div:nth-child(2)""").text

    # reviews =bs4.element.ResultSet -> we have to iterate items in the set so that we can make some chages on each item
    for review in reviews:
        #抓評論者的名字
        gym_dict['name'] = review.find_all('div', class_= "_G7k")[0].text

        #距今有幾天
        pub_date = str(review.find_all('span', class_ ='_Q7k')[0].text)
        if """年""" in pub_date:
            num = int(pub_date.split(" ")[0])
            gym_dict['date'] = num*12*30
        elif """月""" in pub_date:
            num = int(pub_date.split(" ")[0])
            gym_dict['date'] = num*30
        elif """本週""" in pub_date:
            gym_dict['date'] = 7
        else:
            num = int(pub_date.split(" ")[0])
            gym_dict['date'] = num*7

        #抓分數
        score = str(review.find_all('span', class_= "_pxg _Jxg"))
        gym_dict['rating'] = int(re.findall('aria-label="評等：(\d)', score)[0])

        #抓評論
        text = review.find_all('div', class_ ='_ucl')[0].text

        if """(由 Google 翻譯)""" in text:
            gym_dict['comment'] = text.replace("""(由 Google 翻譯) """, "")
            if """(原文)""" in gym_dict['comment']:
                gym_dict['comment'] = gym_dict['comment'].split("""(原文)""")[0]
            else:
                gym_dict['comment'] = gym_dict['comment']
        else:
            gym_dict['comment'] = text
        #去除非文字的表情符號
        gym_dict['comment'] =  ''.join([i for i in gym_dict['comment'] if i.isalpha()])

         #如果要把每次的comment塞進相同的list必須要用.copy() -> urlist.append(urdict.copy())
        gym.append(gym_dict.copy())
    
    return gym

In [ ]:
#!pip install selenium

In [ ]:
#!pip install pymongo

In [5]:
from bs4 import BeautifulSoup
import time
import re 
from pymongo import MongoClient
import concurrent.futures
import threading
from queue import Queue 
import pandas as pd
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.proxy import Proxy
from selenium.webdriver.common.proxy import ProxyType
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
import random

from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import csv


#chrome_path = r"C:\Users\Java\Desktop\專題\chromedriver.exe"

#data = pd.read_csv("/Users/lihsinhuei/project/googlelist")
#store_names = pd.Series.tolist(data['name'])  #把店名取出來變成list

# q = Queue(maxsize=0) #一開始放店名的queue
#q_fail = Queue(maxsize=0) #放失敗的店名
# #for store_name in store_names:
# #   q.put(store_name)
# q.put('悍龍運動空間')
# q.put('world gym 桃園大有店')
# store_names=['悍龍運動空間','健身工廠 Fitness Factory 大有廠','花蓮詠春拳研習館','Curves可爾姿新莊新泰店','新樂舞舞團/樂儛舞蹈工作室','鐵足堂足體養生會館']

# good_proxy_lists = ['89.236.17.108:3128',
#              '188.93.110.26:8080',
#              '185.93.3.123:8080',
#              '151.80.140.233:54566',
#              '133.130.103.208:8080',
#              '35.196.26.166:3128',
#              '192.116.142.153:8080',
#              '188.254.78.108:8080',
#              '89.22.175.42:8080',
#              '203.129.126.226:8080',
#              '154.119.50.246:53281',
#              '194.182.64.67:3128',
#              '216.177.233.181:8080',
#              '61.220.26.97:80',
#              '47.206.51.67:8080',
#              '190.60.4.198:3128',
#              '5.197.231.174:8080']
    


def startDriver():
    thread_amount = 10 #開10個thread
    threads = []
    for i in range(thread_amount):  #開thread
       # ip = good_proxy_lists[random.randint(0, len(good_proxy_lists)-1)]
        t = threading.Thread(target=getStart)
        t.start()
        threads.append(t)   
    for thread in threads:
        thread.join()
        
def getStart(): 
#     chrome_options = webdriver.ChromeOptions()
#     chrome_options.add_argument('--proxy-server=http://%s'%ip)
    #chrome_options.add_argument('--headless')
#     driver = webdriver.Chrome(chrome_options=chrome_options)
    driver = webdriver.Chrome()
#     print(chrome_options.arguments[0])
    n = 1
    client = MongoClient('mongodb://localhost:27017')
    db = client.google_db
    collections = db.google_col
    print("a")
    while not q.empty():
        name = q.get()
        print(name)
        time.sleep(5)
        try:
            #driver.set_page_load_timeout(10)
            print(threading.current_thread())
            print("a")
            driver.get("https://www.google.com.tw")
            searchBar = driver.find_element_by_xpath("""//*[@id="lst-ib"]""")
            print(threading.current_thread())
            print("b")
            #name = q.get()
            searchBar.send_keys(name)
            print(threading.current_thread())
            print("c")
            searchBar.send_keys(Keys.ENTER)
            print(threading.current_thread())
            print("d")
            time.sleep(3)
            comment_cnt = getCnt(driver)
            print(threading.current_thread())
            print("e")
            soup = loadComment(driver,comment_cnt)
            print(threading.current_thread())
            print("f")
            gym = getData(soup)
            print(threading.current_thread())
            print("g")
            db.google_col.insert(gym)
            print(threading.current_thread())
            print("h")
            driver.find_element_by_xpath('//*[@id="gsr"]/g-lightbox/div[2]/div[2]').click()
            print(threading.current_thread())
            print("i")
            driver.find_element_by_xpath("""//*[@id="lst-ib"]""").clear()
            print(threading.current_thread())
            print("j")
            n+=1 
            print(threading.current_thread())
            print("k")
            print("寫入第"+ str(n) +"筆  " + name ) 
            print("l")
#             store_names.remove(name)
            print("m")
        except(InvalidElementStateException,NoSuchElementException,WebDriverException,Exception) as e:
#             store_names.remove(name)
            print("n")
            fail_store_names.append(name)
            print(threading.current_thread())
            print("erro:"+ str(e) + name)
            print(e.with_traceback)
           
            
    driver.close() 
    end = time.time()
    
    start_time = time.localtime(start)
    end_time = time.localtime(end)
    start_project= time.strftime('%Y-%m-%d %H:%M:%S', start_time )
    end_project= time.strftime('%Y-%m-%d %H:%M:%S',end_time )
    print(start_project)
    print(end_project)
    print("總共花了: " + str(end - start))
    client.close()
    
    
    
start = time.time()
q = Queue(maxsize=0)
google_gym =getStoreName()
fail_store_names = []
startDriver()     

# def toDB():    
#     client = MongoClient('mongodb://localhost:27017')
#     db = client.test_db
#     collections = db.test_col
      

    


    

a
韋恩運動中心
a
古銅色健身御戰館（復興本館）
a
ABS Gym健身俱樂部
a
Pace Fitness
a
FUN動健身會館
a
X-LINE27 運動健康俱樂部 台南大灣店
a
無極養生氣功
a
健身工廠FitnessFactory-新莊廠 Xin-Zhuang
a
射箭場
a
敏宇健身房
<Thread(Thread-15, started 9040)>
a
<Thread(Thread-6, started 7688)>
a
<Thread(Thread-10, started 14232)>
a
<Thread(Thread-11, started 16092)>
a
<Thread(Thread-9, started 15620)>
a
<Thread(Thread-8, started 14648)>
a
<Thread(Thread-14, started 9388)>
a
<Thread(Thread-12, started 7896)>
a
<Thread(Thread-13, started 8476)>
a
<Thread(Thread-7, started 8508)>
a
<Thread(Thread-10, started 14232)>
b
<Thread(Thread-10, started 14232)>
c
<Thread(Thread-6, started 7688)>
b
<Thread(Thread-6, started 7688)>
c
<Thread(Thread-8, started 14648)>
b
<Thread(Thread-8, started 14648)>
c
<Thread(Thread-11, started 16092)><Thread(Thread-9, started 15620)>
b

b
<Thread(Thread-9, started 15620)>
c
<Thread(Thread-11, started 16092)>
c
<Thread(Thread-15, started 9040)>
b
<Thread(Thread-15, started 9040)>
c
<Thread(Thread-12, started 7896)>
b
<Thread(Thread-12, 

d:\python3\lib\site-packages\ipykernel_launcher.py:111: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


<Thread(Thread-9, started 15620)>
f
<Thread(Thread-9, started 15620)>
g
<Thread(Thread-11, started 16092)>
f
<Thread(Thread-11, started 16092)>
g
<Thread(Thread-11, started 16092)><Thread(Thread-9, started 15620)>
h

h
<Thread(Thread-6, started 7688)>
h
<Thread(Thread-9, started 15620)>
i
<Thread(Thread-11, started 16092)>
i
<Thread(Thread-6, started 7688)>
i
<Thread(Thread-9, started 15620)>
j
<Thread(Thread-9, started 15620)>
k
寫入第2筆  FUN動健身會館
l
m
斗六全民健身俱樂部
<Thread(Thread-11, started 16092)>
j
<Thread(Thread-11, started 16092)>
k
寫入第2筆  Pace Fitness
l
m
陽光健身中心花蓮館
<Thread(Thread-6, started 7688)>
j
<Thread(Thread-6, started 7688)>
k
寫入第2筆  古銅色健身御戰館（復興本館）
l
m
傭兵戰場 生存遊戲 體驗營
Message: 

拉了2次
<Thread(Thread-10, started 14232)>
f
<Thread(Thread-10, started 14232)>
g
<Thread(Thread-10, started 14232)>
h
<Thread(Thread-10, started 14232)>
i
<Thread(Thread-10, started 14232)>
j
<Thread(Thread-10, started 14232)>
k
寫入第2筆  ABS Gym健身俱樂部
l
m
昕瑜珈(斗六教室)
Message: 

拉了1次
<Thread(Thread-15, started 904

<Thread(Thread-8, started 14648)>
c
<Thread(Thread-13, started 8476)>
a
<Thread(Thread-13, started 8476)>
b
<Thread(Thread-13, started 8476)>
c
<Thread(Thread-8, started 14648)>
d
<Thread(Thread-7, started 8508)>
d
<Thread(Thread-13, started 8476)>
d
<Thread(Thread-15, started 9040)>
a
<Thread(Thread-15, started 9040)>
b
<Thread(Thread-15, started 9040)>
c
<Thread(Thread-12, started 7896)>
a
<Thread(Thread-14, started 9388)>
a
<Thread(Thread-12, started 7896)>
b
<Thread(Thread-14, started 9388)>
b
<Thread(Thread-12, started 7896)>
c
<Thread(Thread-14, started 9388)>
c
<Thread(Thread-8, started 14648)>
e
n
<Thread(Thread-6, started 7688)>
erro:Message: 
世界健身
<built-in method with_traceback of TimeoutException object at 0x1ACAE110>
Curves蘆洲光榮店
<Thread(Thread-15, started 9040)>
d
<Thread(Thread-7, started 8508)>
e
<Thread(Thread-12, started 7896)>
d
<Thread(Thread-13, started 8476)>
e
<Thread(Thread-14, started 9388)>
d
Message: 

拉了2次
<Thread(Thread-9, started 15620)>
f
<Thread(Thread-9,

<Thread(Thread-6, started 7688)>
j
<Thread(Thread-6, started 7688)>
k
寫入第5筆  Anytime Fitness 富民店24小時健身房
l
m
World Gym世界健身俱樂部 新竹林森店
<Thread(Thread-8, started 14648)>
a
<Thread(Thread-8, started 14648)>
b
<Thread(Thread-8, started 14648)>
c
<Thread(Thread-10, started 14232)>
a
<Thread(Thread-10, started 14232)>
b
<Thread(Thread-10, started 14232)>
c
<Thread(Thread-8, started 14648)>
d
Message: 

拉了10次
<Thread(Thread-12, started 7896)>
f
<Thread(Thread-6, started 7688)>
a
<Thread(Thread-10, started 14232)>
d
<Thread(Thread-6, started 7688)><Thread(Thread-12, started 7896)>
b

g
<Thread(Thread-12, started 7896)>
h
<Thread(Thread-12, started 7896)>
i
<Thread(Thread-12, started 7896)>
j
<Thread(Thread-12, started 7896)>
k
寫入第5筆  國立交通大學游泳館
l
m
自由重量健身俱樂部
<Thread(Thread-6, started 7688)>
c
Message: 

拉了1次
<Thread(Thread-13, started 8476)>
f
<Thread(Thread-13, started 8476)>
g
<Thread(Thread-13, started 8476)>
h
<Thread(Thread-13, started 8476)>
i
<Thread(Thread-13, started 8476)>
j
<Thread(Thre

...

<Thread(Thread-8, started 14648)>
e
<Thread(Thread-15, started 9040)>
a
<Thread(Thread-15, started 9040)>
b
<Thread(Thread-15, started 9040)>
c
<Thread(Thread-7, started 8508)>
d
<Thread(Thread-10, started 14232)>
a
<Thread(Thread-10, started 14232)>
b
<Thread(Thread-10, started 14232)>
c
Message: 

拉了6次
<Thread(Thread-14, started 9388)>
f
<Thread(Thread-11, started 16092)>
a
<Thread(Thread-15, started 9040)>Message: 

拉了1次

d
<Thread(Thread-11, started 16092)>
b
<Thread(Thread-14, started 9388)>
g
<Thread(Thread-14, started 9388)>
h
<Thread(Thread-13, started 8476)>
f
<Thread(Thread-11, started 16092)><Thread(Thread-13, started 8476)>
g

c
<Thread(Thread-13, started 8476)>
h
<Thread(Thread-14, started 9388)>
i
<Thread(Thread-13, started 8476)>
i
<Thread(Thread-14, started 9388)>
j
<Thread(Thread-14, started 9388)>
k
寫入第53筆  健身工廠 三多廠
l
m
中華民國消費者文教基金會
<Thread(Thread-13, started 8476)>
j
<Thread(Thread-13, started 8476)>
k
寫入第55筆  Bodyfighter Fitness健身會館
l
m
巨力屋能量王國
<Thread(Thread-10, st

<Thread(Thread-10, started 14232)>
e
<Thread(Thread-15, started 9040)>
e
<Thread(Thread-7, started 8508)>
d
<Thread(Thread-13, started 8476)>
a
<Thread(Thread-11, started 16092)>
e
<Thread(Thread-13, started 8476)>
b
<Thread(Thread-13, started 8476)>
c
<Thread(Thread-14, started 9388)>
a
Message: 

拉了3次
<Thread(Thread-14, started 9388)>
b
<Thread(Thread-14, started 9388)>
c
<Thread(Thread-8, started 14648)>
f
Message: 

拉了1次
<Thread(Thread-8, started 14648)>
g
<Thread(Thread-8, started 14648)>
h
<Thread(Thread-9, started 15620)>
f
<Thread(Thread-9, started 15620)>
g
<Thread(Thread-9, started 15620)>
h
<Thread(Thread-8, started 14648)>
i
<Thread(Thread-7, started 8508)>
e
<Thread(Thread-8, started 14648)>
j
<Thread(Thread-8, started 14648)>
k
寫入第57筆  TRUE YOGA FITNESS桃園會館
l
m
兄弟室內網球場
<Thread(Thread-9, started 15620)>
i
<Thread(Thread-9, started 15620)>
j
<Thread(Thread-9, started 15620)>
k
寫入第55筆  莫爾強身基地（南科 善化 新市）
l
m
Formosa Fitness Gym 健身房
<Thread(Thread-13, started 8476)>
d
<Thread(T

<Thread(Thread-8, started 14648)>
b
<Thread(Thread-8, started 14648)>
c
Message: 

拉了1次
<Thread(Thread-11, started 16092)>
f
<Thread(Thread-11, started 16092)>
g
<Thread(Thread-11, started 16092)>
h
<Thread(Thread-8, started 14648)>
d
<Thread(Thread-7, started 8508)>
a
<Thread(Thread-11, started 16092)>
i
Message: 

拉了2次
<Thread(Thread-11, started 16092)>
j
<Thread(Thread-11, started 16092)>
k
寫入第61筆  高雄私人健身房 I Soul艾斯靈魂
l
m
義大野球村
<Thread(Thread-7, started 8508)>
b
<Thread(Thread-9, started 15620)>
f
<Thread(Thread-7, started 8508)>
c
<Thread(Thread-9, started 15620)>
g
<Thread(Thread-9, started 15620)>
h
<Thread(Thread-9, started 15620)>
i
<Thread(Thread-9, started 15620)>
j
<Thread(Thread-9, started 15620)>
k
寫入第57筆  MYGYM健身俱樂部
l
m
樂天瑜伽 Happy Yoga
<Thread(Thread-14, started 9388)>
e
<Thread(Thread-7, started 8508)>
d
<Thread(Thread-13, started 8476)>
a
<Thread(Thread-13, started 8476)>
b
<Thread(Thread-13, started 8476)>
c
<Thread(Thread-8, started 14648)>
e
<Thread(Thread-13, started

<Thread(Thread-14, started 9388)>
b
<Thread(Thread-7, started 8508)>
g
<Thread(Thread-7, started 8508)>
h
<Thread(Thread-7, started 8508)>
i
<Thread(Thread-7, started 8508)>
j
<Thread(Thread-7, started 8508)>
k
寫入第69筆  魔力夢工廠 親子健身房 Magic Dream work Fitness
l
m
六氧羽球館
<Thread(Thread-14, started 9388)>
c
<Thread(Thread-15, started 9040)>
e
<Thread(Thread-10, started 14232)>
e
<Thread(Thread-14, started 9388)>
d
<Thread(Thread-6, started 7688)>
d
<Thread(Thread-12, started 7896)>
e
<Thread(Thread-8, started 14648)>
a
<Thread(Thread-8, started 14648)>
b
<Thread(Thread-8, started 14648)>
c
Message: 

拉了1次
<Thread(Thread-11, started 16092)>
f
<Thread(Thread-11, started 16092)>
g
<Thread(Thread-11, started 16092)>
h
<Thread(Thread-11, started 16092)>
i
<Thread(Thread-11, started 16092)>
j
<Thread(Thread-11, started 16092)>
k
寫入第63筆  好動專業運動管理
l
m
Curves豐原SOGO店
Message: 

拉了1次
<Thread(Thread-9, started 15620)>
f
<Thread(Thread-9, started 15620)>
g
<Thread(Thread-9, started 15620)>
h
<Thread(Threa

Message: 

拉了2次
<Thread(Thread-7, started 8508)>
f
<Thread(Thread-7, started 8508)>
g
<Thread(Thread-7, started 8508)>
h
<Thread(Thread-7, started 8508)>
i
<Thread(Thread-7, started 8508)>
j
<Thread(Thread-7, started 8508)>
k
寫入第71筆  駿美健身中心
l
m
CLUB S運動曲線S會館
<Thread(Thread-6, started 7688)>
a
<Thread(Thread-11, started 16092)>
a
<Thread(Thread-6, started 7688)>
b
<Thread(Thread-11, started 16092)>
b
<Thread(Thread-6, started 7688)>
c
<Thread(Thread-11, started 16092)>
c
<Thread(Thread-9, started 15620)>
a
<Thread(Thread-9, started 15620)>
b
<Thread(Thread-6, started 7688)>
d
<Thread(Thread-9, started 15620)>
c
Message: 

拉了1次
<Thread(Thread-11, started 16092)>
d
<Thread(Thread-15, started 9040)>
f
<Thread(Thread-15, started 9040)>
g
<Thread(Thread-15, started 9040)>
h
<Thread(Thread-15, started 9040)>
i
<Thread(Thread-15, started 9040)>
j
<Thread(Thread-15, started 9040)>
k
寫入第65筆  花草巷美容spa
l
m
FITBOX 新豐健身房
<Thread(Thread-12, started 7896)>
a
<Thread(Thread-12, started 7896)>
b
<Thread

In [6]:
fail_store_names


['台北市南港運動中心',
 '世界健身',
 '榮美金鬱金香酒店',
 '慢慢來衝浪瑜珈教學營',
 'Air-Gene 空氣基因彈翻床/彈跳床/跳跳床健身育樂中心',
 'Morning Sky']